*Note: This message is a Jupyter Notebook available for [download](https://github.com/bayespy/bayespy-notebooks/blob/master/notebooks/issue34b.ipynb) or for [interactive session](http://mybinder.org/repo/bayespy/bayespy-notebooks/notebooks/issue34b.ipynb). It is a response to BayesPy issue [#34](https://github.com/bayespy/bayespy/issues/34)*.

It's always fun to implement new features that might be useful for someone, so no worries! I implemented `Take` node which should enable you to construct your model, I hope. It is similar to `numpy.take`, so you may want to take a look at that too. I didn't make a new release yet, so you need to use `develop` branch. Use, for instance,

```
pip install https://github.com/bayespy/bayespy/archive/develop.zip
```

If you notice any bugs, please report. You can read the docstring here: https://github.com/bayespy/bayespy/blob/develop/bayespy/inference/vmp/nodes/take.py#L14

Anyway, so I removed `lambda3` and `theta3`, as `theta3` was supposed to be just a deterministic function of `theta1` and `theta2`. Here are the parameters of your model:

In [1]:
from bayespy.nodes import Categorical, Beta, Mixture
lambda1 = Beta([20,5]) 
lambda2 = Beta([[5,20],[20,5]])
theta1 = Categorical(lambda1)
theta2 = Mixture(theta1, Categorical, lambda2)

pi1 = Beta([[[5,20], [20,5]]], plates=(10,2))
pi2 = Beta([[[5,20], [20,5]]], plates=(10,2))
pi3 = Beta([[[5,20], [20,5]]], plates=(10,2))

In order to create `AND` operation for `theta1` and `theta2`, I map the two elements along `pi3` the last plate axis into a 2x2 matrix so that I can use two nested `Mixture` nodes:

In [2]:
from bayespy.nodes import Bernoulli, Take
X1 = Mixture(theta1, Bernoulli, pi1)
X2 = Mixture(theta2, Bernoulli, pi2)
X3 = Mixture(theta1, Mixture, theta2, Bernoulli, Take(pi3, [[0, 0], [0, 1]]))

X1.observe([0,1,0,1,0,1,0,1,0,1])
X2.observe([0,1,0,1,0,1,0,1,0,1])
X3.observe([1,1,1,1,1,1,1,1,1,1])

Basically, you can think of that 2x2 index table as a table which tells how to map values of `theta1` and `theta2` into plate indices for `pi3`. Can't find the words to explain it in a simple way, but it's very simple.. :)

In [3]:
from bayespy.inference import VB
Q = VB(X1, X2, X3, pi1, pi2, pi3, theta2, theta1, lambda1, lambda2)
Q.update(repeat=100)
print(theta1.get_moments()[0])
print(theta2.get_moments()[0])

Iteration 1: loglike=-2.490204e+01 (0.005 seconds)
Iteration 2: loglike=-2.451496e+01 (0.005 seconds)
Iteration 3: loglike=-2.454422e+01 (0.004 seconds)
Converged at iteration 3.
[  1.09166015e-05   9.99989083e-01]
[  2.96225661e-06   9.99997038e-01]


/home/jluttine/Workspace/bayespy/bayespy/inference/vmp/vmp.py:678: UserWarning: Lower bound decreased 2.926577e-02! Bug somewhere or numerical inaccuracy?
  "numerical inaccuracy?" % L_diff)


Oh, it looks like there is a bug in lower bound computation.. I need to take a look.. Anyway, the result looks reasonable for that data.

I'm planning to implement a node which would allow users to construct complex discrete graphs as a part of a model and perform "exact"/non-factorized inference within that set of variables. See [issue #37](https://github.com/bayespy/bayespy/issues/37) for details. But currently, when constructing these discrete variable graphs with BayesPy it factorizes with respect to the nodes. Thus, it is important to understand what you lose: So, let me demonstrate one problem of the mean-field approximation by using a classic XOR example. There are two booleans `v1` and `v2`. A third boolean variable `v3` depends on the others in such a way that it is probably True if only either one of `v1` or `v2` is True. If both or neither of them is True, then `v3` is probably False. Here's the model:

In [4]:
v1 = Categorical([0.5, 0.5])
v2 = Categorical([0.5, 0.5])
v3 = Mixture(v1, Mixture, v2, Categorical,
             [[[0.9,0.1], [0.1,0.9]],
              [[0.1,0.9], [0.9,0.1]]])
v3.observe(1)

Intuitively, because of the symmetry, observing `v3` doesn't give any information about the *marginal* posterior of either `v2` or `v1`. It only couples them. As we observed `v3=1`, there's high probability mass on `(v1=1,v2=0)` and `(v1=0,v2=1)`, and low probability mass on `(v1=0,v2=0)` and `(v1=1,v2=1)`. This probability table can't be represented as a product of two marginals. And because the used VB approximation factorizes as $q(v_1)q(v_2)$, it loses the coupling and captures only one mode. So, if you run the inference algorithm, you'll notice that the result has captured either mode `(v1=1,v2=0)` or `(v1=0,v2=1)`. The mode is chosen randomly, which you'll notice if you re-run the inference several times:

In [5]:
v1.initialize_from_random()
v2.initialize_from_random()
Q = VB(v1, v2, v3)
Q.update(repeat=100, verbose=False)
print(v1.get_moments()[0])
print(v2.get_moments()[0])

[ 0.24894775  0.75105225]
[ 0.75086601  0.24913399]


The true marginals would have been `[0.5, 0.5]` for both `v1` and `v2`. So, whenever you have very strong coupling between different nodes, the approximation tends to capture only one mode and loses the dependency between the variables. This is basic stuff which is good to keep in mind. Maybe it was obvious to you already, but just wanted to point it out. The new `CategoricalGraph` node would get rid of this problem because the node would handle all the dependencies among the variables.

And finally, you asked about getting an incorrect mean (0.813) when sampling a node. Yes, `random()` method of nodes does not give you samples from the generative prior nor does it give you samples from the true posterior. It gives you samples from the current state of the node. As the nodes are factorized, the initial state of the nodes isn't the exact prior distribution because in the prior, the variables are not factorized. Thus, in general, you will not get samples from the true prior, but from an approximation of the prior/posterior, or whatever happens to be the state of the node at the moment. If you add more prior "samples" to you beta distribution, you'll get a better mean estimate:

In [13]:
print(np.mean(Bernoulli(Beta([20, 5]), plates=(100000,)).random()))
print(np.mean(Bernoulli(Beta([200, 50]), plates=(100000,)).random()))
print(np.mean(Bernoulli(Beta([20000, 5000]), plates=(100000,)).random()))
print(np.mean(Bernoulli(Beta([2000000, 500000]), plates=(100000,)).random()))

0.81294
0.79931
0.79989
0.80094


I hope I understood your model correctly this time. Please don't hesitate to ask if you have further questions or comments!